<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [9]</a>'.</span>

# Pipeline 1

In [1]:
from pipeline import Falcon2Linker, SerialAnnotator, T5Converter
import time
from pprint import pprint

In [2]:
linker = Falcon2Linker()
annotator = SerialAnnotator()
converter = T5Converter()

In [3]:
utterance = "Who is the wife of Barack Obama"

In [4]:
linked = linker.link(utterance)
# rules = [1,2,3,4,5,8,9,10,12,13,14]
# linked = process_text_E_R(utterance, rules)
pprint("Linked")
pprint(linked)

pprint("Annotated")
annotated = annotator.annotate(**linked)
pprint(annotated)

pprint("Converted")
converted = converter.preprocess_inputs(**annotated)
pprint(converted)

recieved utterance
0 : ['Who is the wife of Barack Obama']


[reRank_relations]: who is the wife of Barack Obama 
SPARQL Requests Made: 1 
SPARQL Requests Total Time: 0.7170863151550293 
SPARQL Avg Time per Qn: 0.7170863151550293
['Who is the wife of Barack Obama', [['<http://www.wikidata.org/entity/P26>', 'wife'], ['<http://www.wikidata.org/entity/P2848>', 'wife'], ['<http://www.wikidata.org/entity/P140>', 'wife'], ['<http://www.wikidata.org/entity/P451>', 'wife'], ['<http://www.wikidata.org/entity/P600>', 'wife']], [['<http://www.wikidata.org/entity/Q76>', 'Barack obama'], ['<http://www.wikidata.org/entity/Q649593>', 'Barack obama'], ['<http://www.wikidata.org/entity/Q4808526>', 'Barack obama'], ['<http://www.wikidata.org/entity/Q4858106>', 'Barack obama'], ['<http://www.wikidata.org/entity/Q643049>', 'Barack obama']], 0, 0, 0, 0]
'Linked'
{'ents': [{'id': 'Q76',
           'prefix': 'wd:',
           'uri': 'http://www.wikidata.org/entity/Q76'},
          {'id': 'Q649593',
           'prefix': 'wd:',
           'uri': 'http://www.wikidata.org

{'fragments': ['[DEF]',
               'wd:',
               'Q76 Barack Obama',
               '[DEF]',
               'wd:',
               'Q649593 Barack Obama Sr.',
               '[DEF]',
               'wd:',
               'Q4808526 assassination threats against Barack Obama',
               '[DEF]',
               'wd:',
               'Q4858106 Barack Obama Academy',
               '[DEF]',
               'wd:',
               'Q643049 family of Barack Obama',
               '[DEF]',
               'wdt:',
               'P26 spouse',
               '[DEF]',
               'wdt:',
               'P2848 Wi-Fi access',
               '[DEF]',
               'wdt:',
               'P140 religion',
               '[DEF]',
               'wdt:',
               'P451 unmarried partner',
               '[DEF]',
               'wdt:',
               'P600 Wine AppDB ID'],
 'utterance': 'Who is the wife of Barack Obama'}
'Converted'
('Who is the wife of Barack Obama <extra_id_59> <ext

In [5]:
def pipe(utterance, wikisparql):
    linked = linker.link(utterance)
    annotated = annotator.annotate(**linked)
    converted = converter.preprocess(**annotated, wikisparql=wikisparql)
    return linked, annotated, converted

In [6]:
def pipe_batch(utterances, wikisparqls):
  batched = []
  link_batch_start = time.time()
  linked = linker.link_batch(utterances)
  link_batch_end = time.time()
  annotator_time = 0
  converter_time = 0
  for i, single_linked in enumerate(linked):
    s = time.time()
    annotated = annotator.annotate(**single_linked)
    e = time.time()
    try:
      annotator_time += e - s
    except Exception as e:
      print("[Annotator Error]:", e)
      continue
    s = time.time()
    try:
      converted = converter.preprocess(**annotated, wikisparql=wikisparqls[i])
    except Exception as e:
      print("[Converter Error]:", e)
      continue
      
    e = time.time()
    converter_time += e - s
    batched.append([single_linked, annotated, converted])
  print("Link batch time:", link_batch_end - link_batch_start)
  print("Anno batch time:", annotator_time)
  print("Conv batch time:", converter_time)
  return batched

## Data

In [7]:
from pathlib import Path
import pandas as pd
import json

data_path = Path("..") / "t5-for-sparql" / "data" / "lcquad2" / "train.json"

In [8]:
with open(data_path) as f:
  data_dict = json.load(f)

for item in data_dict:
  paraphrased_question = item["paraphrased_question"]
  question = item["question"]
  item["new_question"] = paraphrased_question if len(paraphrased_question) > 2 else question

df = pd.DataFrame.from_dict(data_dict)
df.head()

,NNQT_question,uid,subgraph,template_index,question,sparql_wikidata,sparql_dbpedia18,template,answer,template_id,paraphrased_question,new_question
0,What is the {periodical literature} for {mouth...,19719,simple question right,65,What periodical literature does Delta Air Line...,select distinct ?obj where { wd:Q188920 wdt:P...,select distinct ?obj where { ?statement <http:...,<S P ?O ; ?O instanceOf Type>,[],1,What is Delta Air Line's periodical literature...,What is Delta Air Line's periodical literature...
1,What is {child of} of {husband} of {Ranavalona...,15554,left-subgraph,8,Who is the child of Ranavalona I's husband?,SELECT ?answer WHERE { wd:Q169794 wdt:P26 ?X ....,SELECT ?answer WHERE { ?statement1 <http://www...,C RCD xD . xD RDE ?E,[],5,What is the name of Ranavalona I's husband's c...,What is the name of Ranavalona I's husband's c...
2,Did {Jeff_Bridges} {occupation} {Lane Chandler...,974,boolean double one_hop right subgraph,474,Is it true Jeff_Bridges occupation Lane Chandl...,ASK WHERE { wd:Q174843 wdt:P106 wd:Q1804811 . ...,ASK { ?statement1 <http://www.w3.org/1999/02/...,Ask (ent-pred-obj1` . ent-pred-obj2),[],2,Are Jeff Bridges and Lane Chandler both photog...,Are Jeff Bridges and Lane Chandler both photog...
3,What is {prequel of} of {phase of matter} of {...,15803,right-subgraph,33,What is the pre-requisite of phase matter of G...,SELECT ?answer WHERE { wd:Q675176 wdt:P515 ?X ...,SELECT ?answer WHERE { ?statement1 <http://www...,E REF xF . xF RFG ?G,[],2,What range are the papers at the Monique Genon...,What range are the papers at the Monique Genon...
4,What is <operating income> of <Qantas> ?,27610,center,1907,Which is the operating income for Qantas?,select distinct ?answer where { wd:Q32491 wdt:...,select distinct ?answer where { ?statement <ht...,E REF ?F,[],1.1,[],Which is the operating income for Qantas?


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [9]:
responses = []
batch_qns = []
batch_ans = []
truncated_data = data_dict[12000:]
total_len = len(truncated_data)
last = 0
for i, data in enumerate(truncated_data):
    print("Pipeline iter", i)
    question = data["new_question"]
    answer = data["sparql_wikidata"]
    # try
    #     linked, annotated, converted = pipe(question, answer)
    # except Exception as e:
    #     print(e)
    #     continue
    # responses.append([linked, annotated, converted])
    if len(question) >= 2:
        batch_qns.append(question)
        batch_ans.append(answer)
    if ((i + 1) % 50) == 0 or i == total_len - 1:
        print("[Pipeline1]:", f"Linking {last}-{i}")
        try:
            for linked, annotated, converted in pipe_batch(batch_qns, batch_ans):
                responses.append([linked, annotated, converted])
            batch_qns = []
            batch_ans = []
            last = i
        except Exception as e:
            raise e
            # print(e)
            # continue
        with open(f"../t5-for-sparql/falcon_links/4/link_{i}.json", "w") as f:
            json.dump(responses, f, indent=2, separators=(',',':'))

Pipeline iter 0
Pipeline iter 1
Pipeline iter 2
Pipeline iter 3
Pipeline iter 4
Pipeline iter 5
Pipeline iter 6
Pipeline iter 7
Pipeline iter 8
Pipeline iter 9
Pipeline iter 10
Pipeline iter 11
Pipeline iter 12
Pipeline iter 13
Pipeline iter 14
Pipeline iter 15
Pipeline iter 16
Pipeline iter 17
Pipeline iter 18
Pipeline iter 19
Pipeline iter 20
Pipeline iter 21
Pipeline iter 22
Pipeline iter 23
Pipeline iter 24
Pipeline iter 25
Pipeline iter 26
Pipeline iter 27
Pipeline iter 28
Pipeline iter 29
Pipeline iter 30
Pipeline iter 31
Pipeline iter 32
Pipeline iter 33
Pipeline iter 34
Pipeline iter 35
Pipeline iter 36
Pipeline iter 37
Pipeline iter 38
Pipeline iter 39
Pipeline iter 40
Pipeline iter 41
Pipeline iter 42
Pipeline iter 43
Pipeline iter 44
Pipeline iter 45
Pipeline iter 46
Pipeline iter 47
Pipeline iter 48
Pipeline iter 49
[Pipeline1]: Linking 0-49
1 : ['who is the member of Roberto Baggio sports activities crew and how many fits did they play?']
1 : ['What Campania time zone has 

2 : ['What country is Mojito from?']


[reRank_relations]: what is the mouthpiece that is used for the Deutsche Bahn 
SPARQL Requests Made: 1 
SPARQL Requests Total Time: 0.7181272506713867 
SPARQL Avg Time per Qn: 0.7181272506713867
3 : ['What was the name of the lowest rating International Master Chess Player?']


4 : ['When did  Vladimir Vysotsky marry Lyudmyla Abramova?']
5 : ['Which is the Nickel-Strunz 9th edition (updated 2009) of sphalerite?']


[reRank_relations]: which is the properties for this type of Ambassador 
SPARQL Requests Made: 2 
SPARQL Requests Total Time: 1.452972173690796 
SPARQL Avg Time per Qn: 0.726486086845398
6 : ['Who is the child of Charlemagne and when did he died?']


...

Index error, no label found for
wdt::P134


Index error, no label found for
wdt::P969


Index error, no label found for
wd::Q29960927


Link batch time: 938.1663391590118
Anno batch time: 630.0421817302704
Conv batch time: 0.009049177169799805
Pipeline iter 100
Pipeline iter 101
Pipeline iter 102
Pipeline iter 103
Pipeline iter 104
Pipeline iter 105
Pipeline iter 106
Pipeline iter 107
Pipeline iter 108
Pipeline iter 109
Pipeline iter 110
Pipeline iter 111
Pipeline iter 112
Pipeline iter 113
Pipeline iter 114
Pipeline iter 115
Pipeline iter 116
Pipeline iter 117
Pipeline iter 118
Pipeline iter 119
Pipeline iter 120
Pipeline iter 121
Pipeline iter 122
Pipeline iter 123
Pipeline iter 124
Pipeline iter 125


TypeError: object of type 'NoneType' has no len()

In [ ]:
# qns = ['What is the operating income for Qantas?', 'What is Mary Lou Rettons International Olympic Committee athlete ID.']
# l = linker.link_batch(qns)
# print(l)

In [ ]:
with open("home.json", "w") as f:
  json.dump(responses, f, indent=2 ,separators=(',', ': '))